In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Sabha,LY,2020-10-27 14:43:19,27.04,14.43,85.87,15,0,11.86
1,1,Hamilton,US,2020-10-27 14:43:19,39.18,-84.53,48.20,87,90,3.36
2,2,Araouane,ML,2020-10-27 14:43:19,18.90,-3.53,98.28,9,12,11.45
3,3,Tiznit,MA,2020-10-27 14:42:54,29.58,-9.50,72.70,43,0,9.48
4,4,Kamenka,RU,2020-10-27 14:43:19,51.32,42.77,41.00,69,61,8.88


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# # Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]
# temps = []
# for temp in max_temp:
#     temps.append(max(temp, 0))

In [13]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, 
                                 dissipating=False, 
                                 max_intensity=300, 
                                 point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Sabha,LY,2020-10-27 14:43:19,27.04,14.43,85.87,15,0,11.86
10,10,Port Alfred,ZA,2020-10-27 14:43:23,-33.59,26.89,76.19,73,50,14.67
19,19,Hithadhoo,MV,2020-10-27 14:43:24,-0.60,73.08,83.93,67,75,9.04
20,20,Jalu,LY,2020-10-27 14:43:24,29.03,21.55,85.75,23,100,6.60
22,22,Alvaraes,BR,2020-10-27 14:43:24,-3.22,-64.80,87.80,84,40,4.70
27,27,Saint-Philippe,RE,2020-10-27 14:41:57,-21.36,55.77,77.00,69,0,4.70
29,29,Marakkanam,IN,2020-10-27 14:43:27,12.20,79.95,80.96,78,3,11.12
30,30,Manapparai,IN,2020-10-27 14:43:28,10.60,78.42,84.20,70,20,4.70
31,31,Mahajanga,MG,2020-10-27 14:43:29,-15.72,46.32,84.20,70,20,14.99
32,32,Arraial Do Cabo,BR,2020-10-27 14:43:14,-22.97,-42.03,82.40,69,40,12.75


In [18]:
preferred_cities_df.count()

City_ID       179
City          179
Country       179
Date          179
Lat           179
Lng           179
Max Temp      179
Humidity      179
Cloudiness    179
Wind Speed    179
dtype: int64

In [19]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Sabha,LY,85.87,27.04,14.43,
10,Port Alfred,ZA,76.19,-33.59,26.89,
19,Hithadhoo,MV,83.93,-0.60,73.08,
20,Jalu,LY,85.75,29.03,21.55,
22,Alvaraes,BR,87.80,-3.22,-64.80,
27,Saint-Philippe,RE,77.00,-21.36,55.77,
29,Marakkanam,IN,80.96,12.20,79.95,
30,Manapparai,IN,84.20,10.60,78.42,
31,Mahajanga,MG,84.20,-15.72,46.32,
32,Arraial Do Cabo,BR,82.40,-22.97,-42.03,


In [24]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [28]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]